<a href="https://colab.research.google.com/github/M-Yerro/NLP-ECI2019/blob/master/TP_final_NLP_ECI_2019_Jorge_Han%C3%A9's_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECI 2019 - NLP

## TP del curso "Procesamiento del lenguaje natural mediante redes neuronales"

**Equipo**: Jorge Hané's Team.

**Integrantes**:

-Juan Musmarra

-Nicolás Ojea

-Matías Yerro

**Consigna**:
Uno de los datasets más famosos de Natural Language Inference es el SNLI. En esta tarea se debe responder, dadas dos frases A y B, si B es implicación de A ("entailment"), B es contradictorio con A ("contradiction") o si lo que enuncia B es neutral respecto de A ("neutral"). Se dice que A es la premisa y B es la hipótesis.

En **Gururangan et al., 2018** mostraron que este dataset tiene algunos sesgos, provocados por ejemplo por las heurísticas que tienen los humanos para generar estos pares de frases (A, B). Para ello, desarrollaron un modelo que aún sin observar la premisa A pudiera clasificar el par (A, B) en alguna de las tres clases del dataset.

En este trabajo práctico intentaremos predecir a qué clase pertenece cada una de las hipótesis sin observar la premisa. La idea es replicar los resultados publicados en Gururangan et al., 2018 y mejorarlos si es posible utilizando clasificadores más complejos.

Condiciones de entrega
Además de realizar al menos un envío replicando los resultados del paper antes mencionados, se deberá entregar el código del trabajo práctico junto a un breve informe que explique la idea del modelo implementado y una indicación de cómo ejecutar el programa si fuera necesaria. Ambos elementos se deben enviar al mail germank+eci2019 (así, con el más eci2019, todo junto) arroba gmail punto com. El trabajo podrá hacerse en grupos de hasta 3 personas. Los integrantes deben estar registrados enviando el siguiente formulario: https://forms.gle/gdqZ4WWcU6xucgqu8

**Informe**: 

El trabajo realizado consta de la aplicación de **fastText** sobre el mismo subconjunto del **SNLI** trabajado por Gururangan et al. (2018). Para mayor comodidad tanto en la realización del TP como así en la entrega del mismo, se utilizó el formato Notebook. Tanto el informe como el código se encuentra en el presente archivo.

Se apuntó primero a desarrollar un modelo lo más simple posible, siguiendo la consigna del trabajo, de modo tal que sirviera de "benchmark" a modelos más complejos. 

Los pasos realizados para obtener nuestro mejor resultado fueron los siguientes: 

    -Carga de archivos a partir del Drive

    -Preprocesamiento del texto (pasar todo a minúsculas y eliminar signos de puntuación)
    
    -Incorporación de dos índices a cada oración: el número total de palabras en dicha oración (dato importante según el paper), y el __label__ que permita aplicar fastText de manera supervisada (excepto al conjunto de Test, claramente).
    
    -Instalación de fastText y prueba con un modelo por defecto.
    
    -Batería de modelos en paralelo, para comparar los resultados a partir de la modificación de parámetros.
    
    -Selección del mejor modelo.
    
    -Adaptación de los resultados al formato de Kaggle.
    
Por medio de una parametrización adecuada del fastText hemos logrado 0.68171 en el public leaderboard. El nombre del equipo surgió de la sorpresa de utilizar un código mínimo para obtener lo que a nuestro entender es un resultado aceptable, y de la homofonía del "reduced fastText" con cierto producto muy popular en los infomerciales argentinos de hace unos años.

Además de los desarrollos presentados en este informe, hemos intentado diferentes modelos que no han producido iguales rendimientos. Primero aplicamos CountVectorizer / TfidfVectorizer a las oraciones y usamos esas sparse matrix para la construcción de diferentes modelos (obviando el uso de herramientas como word2vec o fastText). Entre los modelos trabajados podemos mencionar una regresión logística (arrojó .65 en el dev), y varias FFNN (con múltiples capas, funciones de activación, regularización, algoritmos de optimización, etc.). Debido a la importancia de tiempo de procesamiento para comparar las distintas versiones de redes neuronales, se adaptaron los modelos para ser corridos en TPU. Ninguno de estos modelos obtuvo un rendimiento superior al fastText bien parametrizado.

Incluso intentamos utilizar los embeddings producto del fastText como imput para esas FFNN, pero aquí nuevamente ninguna logró superar los valores alcanzados por fastText. Quizás modelos más complejos como LSTM o CNN podrían trabajar con estos embeddings y superar la clasificación mediante fastText sólo, pero nuestros resultados en este sentido con FFNN no han sido superadores.


## Carga de archivos a partir del Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import pandas as pd

data_folder = 'gdrive/My Drive/NLP/'

labels_dev = pd.read_csv(data_folder + 'snli_1.0_dev_gold_labels.csv')
labels_train = pd.read_csv(data_folder + 'snli_1.0_train_gold_labels.csv')

data_train = pd.read_json(data_folder + 'snli_1.0_train_filtered.jsonl', lines=True)
data_dev = pd.read_json(data_folder + 'snli_1.0_dev_filtered.jsonl', lines=True)
data_test = pd.read_json(data_folder + 'snli_1.0_test_filtered.jsonl', lines=True)

##Preprocesamiento del texto

In [0]:
data_train["sentence2"] = data_train["sentence2"].str.lower()
data_dev["sentence2"] = data_dev["sentence2"].str.lower()
data_test["sentence2"] = data_test["sentence2"].str.lower()

import string
Tabel = str.maketrans('', '', string.punctuation)
data_train["sentence2"] = data_train["sentence2"].str.translate(Tabel)
data_dev["sentence2"] = data_dev["sentence2"].str.translate(Tabel)
data_test["sentence2"] = data_test["sentence2"].str.translate(Tabel)

## Creación de los dataframes a utilizar

In [0]:
df_train = pd.concat([data_train['sentence2'], labels_train['gold_label']], axis=1, keys=['sentence', 'label'])
df_train['sentence_label'] = df_train.apply(lambda row: '__label__'+row['label']+' '+row['sentence'], axis=1)
df_train['n_words'] = df_train['sentence'].str.split().apply(len)
# df_train.head()

In [0]:
df_dev = pd.concat([data_dev['sentence2'], labels_dev['gold_label']], axis=1, keys=['sentence', 'label'])
df_dev['sentence_label'] = df_dev.apply(lambda row: '__label__'+row['label']+' '+row['sentence'], axis=1)
df_dev['n_words'] = df_dev['sentence'].str.split().apply(len)
# df_dev.head()

In [0]:
df_test = pd.concat([data_test['sentence2'], data_test['pairID']], axis=1, keys=['sentence', 'pairID'])
df_test['n_words'] = df_test['sentence'].str.split().apply(len)
# df_test.head()

# Reduce(d) (Fat) fastText

## Creación de los txt para fastText

In [0]:
import numpy as np

np.savetxt(data_folder+'train.txt', df_train['sentence_label'].values, fmt='%s')
np.savetxt(data_folder+'dev.txt', df_dev['sentence_label'].values, fmt='%s')

## Instalación de fastText. Primer modelo con parámetros por defecto.

In [0]:
!pip install fasttext
import fasttext

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.6f}".format(1, p))
    print("R@{}\t{:.6f}".format(1, r))

jorge_hane = fasttext.train_supervised(data_folder+'train.txt')
print_results(*jorge_hane.test(data_folder+'dev.txt'))

N	9842
P@1	0.645499
R@1	0.645499


## Batería de prueba de parámetros para mejorar el modelo.

PARÁMETROS POR DEFECTO ([FUENTE](https://pypi.org/project/fasttext/#train-supervised-parameters))
```
input             # training file path (required)
lr                # learning rate [0.1]
dim               # size of word vectors [100]
ws                # size of the context window [5]
epoch             # number of epochs [5]
minCount          # minimal number of word occurences [1]
minCountLabel     # minimal number of label occurences [1]
minn              # min length of char ngram [0]
maxn              # max length of char ngram [0]
neg               # number of negatives sampled [5]
wordNgrams        # max length of word ngram [1]
loss              # loss function {ns, hs, softmax, ova} [softmax]
bucket            # number of buckets [2000000]
thread            # number of threads [number of cpus]
lrUpdateRate      # change the rate of updates for the learning rate [100]
t                 # sampling threshold [0.0001]
label             # label prefix ['__label__']
verbose           # verbose [2]
pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []
```

In [0]:
#Esta celda contenía varios fasttext.train_supervised con diferentes parámetros con el objetivo de mejorar el resultado alcanzado con el modelo por defecto.
#Se los ha eliminado con el objetivo de mejorar la legibilidad del trabajo.

Jorge Hané, First of His Name, King of the Andals and the First Men, Lord of the Six Kingdoms, and Protector of the Realm
N	9842
P@1	0.678013
R@1	0.678013
Hané the second
N	9842
P@1	0.675574
R@1	0.675574
Hané the third
N	9842
P@1	0.676590
R@1	0.676590
Hané Hané Hané Hané
N	9842
P@1	0.676692
R@1	0.676692


## El mejor modelo por ahora.

In [0]:
jorge_hane_best = fasttext.train_supervised(data_folder+'train.txt', lr=0.11, dim=170, ws=5, wordNgrams=2, loss='ova', epoch=7)
print_results(*jorge_hane_best.test(data_folder+'dev.txt'))

N	9842
P@1	0.681467
R@1	0.681467


## Guardo predicciones de jorge_hane_best en un csv para subir a kaggle

In [0]:
def hane_predict(sentence, fasttext_model=jorge_hane_best):
  return fasttext_model.predict(sentence)[0][0].replace('__label__', '')

df_test['jorge_hane_best_predicted_label'] = df_test['sentence'].apply(hane_predict)

df_para_subir_con_jorge_hane_best_predicted_labels = pd.concat([df_test['pairID'], df_test['jorge_hane_best_predicted_label']], axis=1, keys=['pairID', 'gold_label'])
print(df_para_subir_con_jorge_hane_best_predicted_labels.head())

df_para_subir_con_jorge_hane_best_predicted_labels.to_csv(data_folder+'result.csv', index=False)


                pairID     gold_label
0  2677109430.jpg#1r1n  contradiction
1  2677109430.jpg#1r1e        neutral
2  2677109430.jpg#1r1c  contradiction
3  6160193920.jpg#4r1n        neutral
4  6160193920.jpg#4r1e        neutral


# Otras locuras...

## Chequeo si pairID es predictor perfecto

In [0]:
def check_label_in_last_char_of_pair_id(df=labels_train):
  for index, row in df.iterrows():
     if row['pairID'][-1] == 'n' and row['gold_label'] != 'neutral' or \
      row['pairID'][-1] == 'e' and row['gold_label'] != 'entailment' or \
      row['pairID'][-1] == 'c' and row['gold_label'] != 'contradiction':
        print('El label no coincide con la ultima letra del pairID para el indice: {}'.format(index))
        return False
  return True
print(check_label_in_last_char_of_pair_id())

## Diferencias entre 2 result.csv

In [0]:
df_res = pd.read_csv(data_folder+'result.csv', index_col=0)
df_res_old = pd.read_csv(data_folder+'result_old.csv', index_col=0)
df_all = pd.concat([df_res['gold_label'], df_res_old['gold_label']], axis=1, keys=['label_res', 'label_res_old'])
df_all.head()
df.query('label_res != label_res_old')